In [1]:
import pandas as pd
import os

# to save time, only decomment below 2 if necessary:
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering

from tqdm import tqdm
from datetime import datetime

In [23]:
df=pd.read_feather('r2-QC393-1251227.feather')
# df=pd.read_stata("d://r2.dta")
CPCleanedUnique=list(df.CPCleaned.unique())

In [67]:
# func def.:

def assign(q,orig:bool=False, cp="delete"):
    global df
    if orig==True:
        df.loc[df.Q==q,"CPCleaned"]=cp
    else:
        df.loc[df.QCleaned==q,"CPCleaned"]=cp
        
def getOriginalQ(cleanedQ):
    global df
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

# decomment only needed:
# def groupSent(sentences, threshold=0.75):
#     model = SentenceTransformer('all-MiniLM-L6-v2')
#     embeddings = model.encode(sentences,show_progress_bar=False)
#     clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
#     clustering.fit(embeddings)
#     clusters = {}
#     for sentence_id, cluster_id in enumerate(clustering.labels_):
#         if cluster_id not in clusters:
#             clusters[cluster_id] = []
#         clusters[cluster_id].append(sentences[sentence_id])
#     return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

# with open("grpdQ.txt","r") as txt:
#     grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

def collapse(qcleaned, country):
    global df
    return df[(df.QCleaned==qcleaned) & (df.Country==country)].Q.value_counts()

def tab(cpnum=None,cp=None):
    assert cpnum!=None or cp!=None, "pass in at least 1 param."
    if cpnum!=None:
        cp=CPCleanedUnique[cpnum]
    print(cp)
    global df
    filtered_df=df[df['CPCleaned']==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def tabOcp(cporig=None):
    global df
    filtered_df=df[df['CP']==cporig]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def tabOcpQ(cporig=None):
    global df
    filtered_df=df[df['CP']==cporig]
    return pd.crosstab(filtered_df['Country'], filtered_df['Q'],margins=True).transpose()


def discardYear(qcleaned, country, year:int=2021):
    global df
    df.loc[(df.QCleaned==qcleaned) & (df.Country==country) & (df.AssesmentDate.str[-4:]==str(year)),"CPCleaned"]="delete"

def exportStata(path=datetime.now().strftime("%Y%m%d%H%M")):
    df.to_stata(path, version=118)



---
# debug:

In [72]:
tab(0)

other grounds


Country,Bangladesh,Cambodia,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,
are a worker's family responsibilities a factor in the employer's decisions regarding termination or retirement of workers?,0,0,0,0,0,27,27
are disabled persons who apply for work evaluated according to their ability to perform the job?,0,2237,0,0,119,1802,4158
are family responsibilities a factor in decisions regarding opportunities for promotion or access to training?,0,0,0,0,0,27,27
are family responsibilities a factor in decisions regarding pay?,0,0,0,0,0,54,54
are hiv/aids tests required at hiring?,0,0,0,0,0,1802,1802
are workers harassed on the basis of their family responsibilities?,0,0,0,0,0,27,27
compliance on hiring disabled workers,0,3,851,474,0,0,1328
do recruitment materials such as job announcements or job application forms refer to age in a manner that could discourage older workers from applying?,0,0,0,0,0,721,721
"does the employer allow workers with aids-related illnesses to work for as long as medically fit in available, appropriate work?",0,0,0,0,2,1802,1804


In [73]:
tab(12)

social security and other benefits


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are the workers' participation and welfare funds used and distributed as required?,584,0,0,0,0,0,0,584
collect and deposit workers' contributions into the fund,584,0,0,0,0,0,0,584
do workers receive required distributions from the workers' participation fund?,18,0,0,0,0,0,0,18
does the employer collect and forward workers' contributions to social insurance funds on time?,0,0,0,0,0,120,1802,1922
does the employer comply with other wage payments?,18,0,0,0,0,0,0,18
does the employer comply with requirements when workers have occupational accidents and diseases?,0,0,0,0,0,0,1802,1802
does the employer correctly calculate and send the employer contribution to the social security corporation?,0,0,0,0,474,0,0,474
"does the employer correctly calculate, collect, and forward workers' contributions to the social security corporation?",0,0,0,0,474,0,0,474
does the employer forward workers contributions to ofatma?,0,0,161,0,0,0,0,161


# append:

- `p2["index"].nunique()-len(df)` : `-16266`

- `len(df),len(p2)` : `(1251227, 1234961)`

In [46]:
dfl=CPCleanedUnique[:25]
p2l=CPCleanedUnique[25:]
p2=pd.read_csv("d://p2.csv")
p2.drop(["year","index"],axis=1, inplace=True)

In [53]:
set(p2.columns)==set(df.columns)

True

In [57]:
dff=pd.concat([df[df.CPCleaned.isin(dfl)], p2[p2.CPCleaned.isin(p2l)]], ignore_index=True)

In [59]:
dff.to_stata("d://concat.dta",version=118)

In [60]:
dff.to_feather("d://concat.feather")

In [64]:
dff[dff.CPCleaned=="strikes"].QCleaned.value_counts()

employer punished workers for striking       6497
employer prevented workers from striking     6212
employer replaced striking workers           6212
employer called police to break up strike    6209
Name: QCleaned, dtype: int64